## **Email Alert**

### **Using live video**

In [2]:
import cv2

# Open the default camera
cap = cv2.VideoCapture(0)

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'X264')
out = cv2.VideoWriter(r"C:\Users\Lenovo\OneDrive\Desktop\Final Code\Live_Video\LIVE.mp4", fourcc, 20.0, (640, 480))

while cap.isOpened():
    # Capture frame-by-frame
    ret, frame = cap.read()
    if ret:
        # Display the resulting frame
        cv2.imshow('frame', frame)
        
        # Write the frame to the output file
        out.write(frame)
        
        # Press 'q' to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

# Release everything when done
cap.release()
out.release()
cv2.destroyAllWindows()


In [3]:
from playsound import playsound
       

In [12]:
from tensorflow.keras.models import load_model

# Load the saved model from disk
LRCN_model = load_model("LRCN_Model.h5")

In [13]:
# Specify the height and width to which each video frame will be resized in our dataset.
IMAGE_HEIGHT , IMAGE_WIDTH = 64, 64

# Specify the number of frames of a video that will be fed to the model as one sequence.
SEQUENCE_LENGTH = 20

# Specify the list containing the names of the classes used for training. Feel free to choose any set of classes.
CLASSES_LIST = ["NonViolence","Violence","Fire"]


In [14]:
input_video_file_path=r"C:\Users\Lenovo\OneDrive\Desktop\Final Code\Live_Video\LIVE.mp4"

In [15]:
sound_file_path = r"C:\Users\Lenovo\OneDrive\Desktop\Final Code\emergency-alarm-with-reverb-29431.mp3"

In [16]:
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import cv2
import numpy as np
def predict_single_action(video_file_path, SEQUENCE_LENGTH):
    '''
    This function will perform single action recognition prediction on a video using the LRCN model.
    Args:
    video_file_path:  The path of the video stored in the disk on which the action recognition is to be performed.
    SEQUENCE_LENGTH:  The fixed number of frames of a video that can be passed to the model as one sequence.
    '''

    # Initialize the VideoCapture object to read from the video file.
    video_reader = cv2.VideoCapture(video_file_path)

    # Get the width and height of the video.
    original_video_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_video_height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Declare a list to store video frames we will extract.
    frames_list = []
    
    # Initialize a variable to store the predicted action being performed in the video.
    predicted_class_name = ''

    # Get the number of frames in the video.
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate the interval after which frames will be added to the list.
    skip_frames_window = max(int(video_frames_count/SEQUENCE_LENGTH),1)

    # Iterating the number of times equal to the fixed length of sequence.
    for frame_counter in range(SEQUENCE_LENGTH):

        # Set the current frame position of the video.
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)

        # Read a frame.
        success, frame = video_reader.read() 

        # Check if frame is not read properly then break the loop.
        if not success:
            break

        # Resize the Frame to fixed Dimensions.
        resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
        
        # Normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1.
        normalized_frame = resized_frame / 255
        
        # Appending the pre-processed frame into the frames list
        frames_list.append(normalized_frame)

    # Passing the  pre-processed frames to the model and get the predicted probabilities.
    predicted_labels_probabilities = LRCN_model.predict(np.expand_dims(frames_list, axis = 0))[0]

    # Get the index of class with highest probability.
    predicted_label = np.argmax(predicted_labels_probabilities)

    # Get the class name using the retrieved index.
    predicted_class_name = CLASSES_LIST[predicted_label]
#     confidence = format(predicted_labels_probabilities[predicted_label])
#     confidence_percent = round(confidence * 100, 2)
    confidence = predicted_labels_probabilities[predicted_label]
    confidence_float = float(confidence)
    confidence_percent = round(confidence_float * 100, 2)
    
    # If the predicted class is Violence, send an email with the video attachment.
    if predicted_class_name == 'Violence':
        print("Sending email...")

        # email credentials
        EMAIL_ADDRESS = 'finalyrproject23@gmail.com'
        EMAIL_PASSWORD = 'gubjkajaztkdmtty'
        RECEIVER_ADDRESS = 'ridha.isha20@gmail.com,maiseridha@gmail.com,'

        # message details
        subject = 'Warning: Violence detected in video'
        body = f'A video containing violent content has been detected. Confidence: {confidence_percent}%'

        # create message
        message = MIMEMultipart()
        message['From'] = EMAIL_ADDRESS
        message['To'] = RECEIVER_ADDRESS
        message['Subject'] = subject

        # add message body
        message.attach(MIMEText(body, 'plain'))

        # attach video file
        attachment = open(input_video_file_path, 'rb')
        video_mime = MIMEBase('application', 'octet-stream')
        video_mime.set_payload((attachment).read())
        encoders.encode_base64(video_mime)
        video_mime.add_header('Content-Disposition', "attachment; filename= %s" % os.path.basename(input_video_file_path))
        message.attach(video_mime)
        
        to_addrs = RECEIVER_ADDRESS.split(',')
        # send email
        session = smtplib.SMTP('smtp.gmail.com',587)
        session.starttls()
        session.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
        text = message.as_string()
        session.sendmail(EMAIL_ADDRESS,to_addrs , text)
        session.quit()

        print("Email sent.")
        playsound(sound_file_path)
        print("Alert sound played.")
    
    elif predicted_class_name == 'Fire':
        print("Sending email...Fire")

        # email credentials
        EMAIL_ADDRESS = 'finalyrproject23@gmail.com'
        EMAIL_PASSWORD = 'gubjkajaztkdmtty'
        RECEIVER_ADDRESS = 'ridha.isha20@gmail.com,maiseridha@gmail.com'
        
        # message details
        subject = 'Warning: Fire detected in video'
        body = f'A video containing Fire has been detected. Confidence: {confidence_percent}%'

        # create message
        message = MIMEMultipart()
        message['From'] = EMAIL_ADDRESS
        message['To'] = RECEIVER_ADDRESS
        message['Subject'] = subject

        # add message body
        message.attach(MIMEText(body, 'plain'))

        # attach video file
        attachment = open(input_video_file_path, 'rb')
        video_mime = MIMEBase('application', 'octet-stream')
        video_mime.set_payload((attachment).read())
        encoders.encode_base64(video_mime)
        video_mime.add_header('Content-Disposition', "attachment; filename= %s" % os.path.basename(input_video_file_path))
        message.attach(video_mime)
        
        to_addrs = RECEIVER_ADDRESS.split(',')

        
        # send email
        session = smtplib.SMTP('smtp.gmail.com',587)
        session.starttls()
        session.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
        text = message.as_string()
        session.sendmail(EMAIL_ADDRESS,to_addrs , text)
        session.quit()

        print("Email sent.")
        playsound(sound_file_path)
        print("Alert sound played.")
        
    else:
        print(f'Action Predicted: {predicted_class_name}\nConfidence: {predicted_labels_probabilities[predicted_label]}')



In [17]:
predict_single_action(input_video_file_path, SEQUENCE_LENGTH)

1/1 [==============================] - 1s 886ms/step
Sending email...



    Error 305 for command:
        open "C:\Users\Lenovo\AppData\Local\Temp\PSa4uhnjv9.mp3"
    Cannot specify extra characters after a string enclosed in quotation marks.

    Error 305 for command:
        close "C:\Users\Lenovo\AppData\Local\Temp\PSa4uhnjv9.mp3"
    Cannot specify extra characters after a string enclosed in quotation marks.
Failed to close the file: "C:\Users\Lenovo\AppData\Local\Temp\PSa4uhnjv9.mp3"


Email sent.


PlaysoundException: 
    Error 305 for command:
        open "C:\Users\Lenovo\AppData\Local\Temp\PSa4uhnjv9.mp3"
    Cannot specify extra characters after a string enclosed in quotation marks.

### **Using saved videos**

In [ ]:
input_video_file_path=(r"C:\Users\Lenovo\OneDrive\Desktop\Final Code\test_dataset\NV2.mp4")

predict_single_action(input_video_file_path, SEQUENCE_LENGTH)

1/1 [==============================] - 0s 32ms/step
Action Predicted: NonViolence
Confidence: 0.9948554039001465


In [ ]:
input_video_file_path=r"C:\Users\Lenovo\OneDrive\Desktop\Final Code\test_dataset\Fire2.mp4"

predict_single_action(input_video_file_path, SEQUENCE_LENGTH)

1/1 [==============================] - 0s 40ms/step
Sending email...Fire


In [ ]:
input_video_file_path=r"C:\Users\Lenovo\OneDrive\Desktop\Final Code\test_dataset\V3.mp4"

predict_single_action(input_video_file_path, SEQUENCE_LENGTH)

1/1 [==============================] - 0s 27ms/step
Sending email...
Email sent.
Alert sound played.
